In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import trapz
from scipy.signal import find_peaks
from lets_plot import *
import pingouin as pg

LetsPlot.setup_html()

# sample information

In [137]:
# importing dataframe
df = pd.read_excel("C:/Users/bvenn/OneDrive/Desktop/Publikation/Dataframe/Beta_Ala_Dataframe.xlsx")
df

,id,group,age,sex,weight_kg,hight_m,bmi
0,25778,Beta Ala,25,w,57,1.63,21.454000
1,54145,Beta Ala,21,w,72,1.72,24.337000
2,15073,Beta Ala,24,m,69,1.82,20.380000
3,20438,Beta Ala,25,m,76,1.80,23.457000
4,49830,Beta Ala,26,w,82,1.68,29.053000
5,58046,Beta Ala,23,m,77,1.85,22.498000
6,26107,Beta Ala,22,w,65,1.70,22.491000
7,44858,Beta Ala,26,w,53,1.64,19.706000
8,59289,Beta Ala,21,m,75,1.78,23.671254
9,93672,Beta Ala,24,m,82,1.77,26.173833


## Import isokinetic raw data from github repository

In [5]:
iso = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/isokinet_dataframe.csv?token=BGUXDRDMA33EFDADTDHQ32DF4XMJY")

# format data
iso['id'] = iso['id'].astype('int64')
iso['measurement'] = iso['measurement'].astype('category')
iso['experiment'] = iso['experiment'].astype('category')
iso['group'] = iso['group'].astype('category')

# tranform data to its units
iso['pos_1/10_deg/s_or_mm'] = iso['pos_1/10_deg/s_or_mm'] / 10
iso['torque_1/10nm_or_force_n'] = iso['torque_1/10nm_or_force_n'] / (-10)
iso['speed_1/10_deg/sec_or_1/10_mm/s'] = iso['speed_1/10_deg/sec_or_1/10_mm/s'] / 10


iso.rename(columns={'pos_1/10_deg/s_or_mm': 'pos_deg_or_mm', 'torque_1/10nm_or_force_n': 'torque_nm',
                   'speed_1/10_deg/sec_or_1/10_mm/s': 'speed_deg/s_or_mm/s'}, inplace=True)
iso.head()

,id,measurement,time_ms,pos_deg_or_mm,torque_nm,speed_deg/s_or_mm/s,torque_without_comp,rep,set,torque_dyn,force_right,force_left,experiment,group
0,15073,warmup,5,12.0,-3.0,0.0,293,1,1,30,0,0,pretest,beta_ala
1,15073,warmup,10,12.0,-3.0,0.5,293,1,1,30,0,0,pretest,beta_ala
2,15073,warmup,15,12.0,-2.3,4.0,293,1,1,23,0,0,pretest,beta_ala
3,15073,warmup,20,12.1,-0.0,18.3,293,1,1,0,0,0,pretest,beta_ala
4,15073,warmup,25,12.1,11.3,23.5,293,1,1,-113,0,0,pretest,beta_ala


## Calculate Peak Torque for each participant in pre- and post-test
- peak torque maximum torque acomplished
- colum abbruchkriterium = 50% of maximal torque value. At three consequtive repetitions below that value, sufficient fatigue was induced and the protocol was terminated. pre = pretest, post = posttest

In [7]:
# create dataframe with torque values
pt_results = pd.DataFrame(columns = ['id', 'group', 'peak_torque_pre', 'abbruchkriterium_pre', 'peak_torque_post',
                                     'abbruchkriterium_post'])

probanden_ids = iso['id'].unique()

for proband_id in probanden_ids:
    proband_data = iso[iso['id'] == proband_id]
    
    # add group information
    group = proband_data['group'].iloc[0]
    
    # filter for pre and post measurements
    proband_data_pre = proband_data[proband_data['experiment'] == 'pretest']
    proband_data_post = proband_data[proband_data['experiment'] == 'posttest']
    
    # filter for concentric reps. every second rep
    kon_pre = proband_data_pre.query('measurement == "pt" and rep % 2 == 0')
    kon_post = proband_data_post.query('measurement == "pt" and rep % 2 == 0')
    
    # calculate termination criterion
    # pretest
    
    peak_torque_pre = kon_pre['torque_nm'].max()
    abort_pre = peak_torque_pre / 2
    
    # posttest
    
    peak_torque_post = kon_post['torque_nm'].max()

    # termination at 50% max torque
    abort_post = peak_torque_post / 2
    
    ergebnis_row = pd.DataFrame({
        'id': [proband_id],
        'group': [group],
        'peak_torque_pre': [peak_torque_pre],
        'abbruchkriterium_pre': [abort_pre],
        'peak_torque_post': [peak_torque_post],
        'abbruchkriterium_post': [abort_post]
        
    })
    
    # add results to dataframe
    pt_results = pd.concat([pt_results, ergebnis_row], ignore_index = True)
    
pt_results

C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\2043861747.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pt_results = pd.concat([pt_results, ergebnis_row], ignore_index = True)


,id,group,peak_torque_pre,abbruchkriterium_pre,peak_torque_post,abbruchkriterium_post
0,15073,beta_ala,249.5,124.75,269.0,134.50
1,20438,beta_ala,306.8,153.40,291.0,145.50
2,25778,beta_ala,163.2,81.60,155.7,77.85
3,26107,beta_ala,201.5,100.75,194.0,97.00
4,27351,beta_ala,162.8,81.40,166.5,83.25
5,28514,placebo,121.2,60.60,139.2,69.60
6,93672,beta_ala,294.5,147.25,338.3,169.15
7,33468,placebo,156.5,78.25,171.0,85.50
8,39337,placebo,296.7,148.35,261.5,130.75
9,39750,placebo,139.2,69.60,131.3,65.65


In [8]:
pt_results_l = pd.melt(pt_results.drop(columns=['abbruchkriterium_pre', 'abbruchkriterium_post']), id_vars = ['id', 'group'], value_name='torque', var_name = 'measurement')
pt_results_l.head()

,id,group,measurement,torque
0,15073,beta_ala,peak_torque_pre,249.5
1,20438,beta_ala,peak_torque_pre,306.8
2,25778,beta_ala,peak_torque_pre,163.2
3,26107,beta_ala,peak_torque_pre,201.5
4,27351,beta_ala,peak_torque_pre,162.8


In [10]:
# Peak Torque group differences

(
    ggplot(pt_results_l, aes(x='measurement', y='torque'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Peak Torque at pre and post test - seperated by groups.')
    + ylab('Peak Torque [Nm]')
)

In [11]:
pt_stat = pg.mixed_anova(dv='torque',between='group', within='measurement', subject='id', data=pt_results_l, correction='auto', effsize='np2')
pt_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,13313.340833,1,22,13313.340833,1.684440,0.207779,0.071120,NaN
1,measurement,189.607500,1,22,189.607500,1.118742,0.301666,0.048391,1.0
2,Interaction,23.520000,1,22,23.520000,0.138775,0.713066,0.006268,NaN


### Numer of Reps done to termination criterion

In [24]:
# Sets done = Number of sets done last set included
# total_reps = total sum of reps
# to open this file download the excel file and copy the path in the bracets after pd.read_excel(...)

absolvierte_reps = pd.read_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Zusammenfassung Arbeitssätze\Absolvierte_reps.xlsx")
absolvierte_reps

,id,group,sets_done_pre,reps_done_last set_pre,total_reps_pre,sets_done_post,reps_done_last_set_post,total_reps_post,Unnamed: 8,letzte_peaks_1_pre,letzte_peaks_2_pre,letzte_peaks_3_pre,Mean_last_peaks_pre,letzte_peaks_1_post,letzte_peaks_2_post,letzte_peaks_3_post,Mean_last_peaks_post
0,15073,beta_ala,5,15,75,6,13,88,NaN,127.5,122.3,119.3,123.033333,125.7,123.8,115.5,121.666667
1,20438,beta_ala,2,12,27,7,13,103,NaN,146.7,146.3,143.7,145.566667,140.0,122.3,123.0,128.433333
2,25778,beta_ala,4,15,60,2,14,29,NaN,75.8,77.0,80.7,77.833333,70.5,68.7,62.3,67.166667
3,26107,beta_ala,8,15,120,6,14,89,NaN,101.7,93.8,87.0,94.166667,95.0,94.5,87.0,92.166667
4,27351,beta_ala,2,14,29,1,14,14,NaN,56.3,75.0,72.8,68.033333,74.3,71.3,65.0,70.200000
5,28514,placebo,7,15,105,7,9,99,NaN,57.8,63.8,65.7,62.433333,57.5,53.7,62.3,57.833333
6,33468,placebo,3,15,45,2,14,29,NaN,70.5,74.3,72.8,72.533333,81.8,75.8,80.3,79.300000
7,39337,placebo,3,14,44,4,15,60,NaN,149.3,146.3,141.5,145.700000,109.5,117.5,117.5,114.833333
8,39750,placebo,2,15,30,4,12,57,NaN,71.3,65.3,60.5,65.700000,71.7,56.7,60.5,62.966667
9,44858,beta_ala,3,14,44,4,13,58,NaN,72.0,63.8,61.5,65.766667,80.0,68.7,73.5,74.066667


In [25]:

absolvierte_reps_l = pd.melt(absolvierte_reps[['total_reps_pre', 'total_reps_post', 'id', 'group']], 
                             id_vars=['id', 'group'], var_name = 'measurement', value_name='reps done')

In [27]:
(
    ggplot(absolvierte_reps_l, aes(x='measurement', y='reps done'))
    + geom_boxplot(aes(color='group', group='group'))
    + labs(title='Completed reps until termation')
    + ylab('Reps total')
)

In [29]:
# to open this file download the excel file Absolvierte_reps and copy the path in the bracets after pd.read_excel(...)

rep_stat = pd.read_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Zusammenfassung Arbeitssätze\Absolvierte_reps.xlsx", 
                         sheet_name = 'Sheet3')

reps_stat = pg.mixed_anova(dv='reps',between='group', within='experiment', subject='id', data=rep_stat, correction='auto', effsize='np2')
reps_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,784.083333,1,22,784.083333,0.739588,0.399068,0.032524,NaN
1,experiment,2.083333,1,22,2.083333,0.006085,0.938527,0.000277,1.0
2,Interaction,108.000000,1,22,108.000000,0.315458,0.580028,0.014136,NaN


## Calculate work done in pre and post test as integral over the complete range of motion.


In [33]:
# extract ids
probanden_ids = iso['id'].unique()

# create data-frame
total_work_done = pd.DataFrame(columns=['id', 'experiment', 'group', 'work_done'])

for proband_id in probanden_ids:
    proband_data = iso[iso['id'] == proband_id]
    
    # group information
    group = proband_data['group'].iloc[0]
    
    # filter for pre and post-test
    proband_data_pre = proband_data[proband_data['experiment'] == 'pretest'].copy()
    proband_data_post = proband_data[proband_data['experiment'] == 'posttest'].copy()

    # adjust time column
    proband_data_pre.loc[:, 'time_ms'] = proband_data_pre.index * 5 + 5
    proband_data_post.reset_index(drop=True, inplace=True)
    proband_data_post.loc[:, 'time_ms'] = proband_data_post.index * 5 + 5

    # calculate position change
    proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['pos_deg_or_mm'].diff()
    proband_data_post['displacement_deg_or_mm'] = proband_data_post['pos_deg_or_mm'].diff()
    proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['displacement_deg_or_mm'].where(proband_data_pre['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
    proband_data_post['displacement_deg_or_mm'] = proband_data_post['displacement_deg_or_mm'].where(proband_data_post['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
    
    # sum position change as total distance
    gesamtstrecke_mm_pre = proband_data_pre['displacement_deg_or_mm'].abs().sum()
    gesamtstrecke_m_pre = gesamtstrecke_mm_pre / 1000  # Auf Meter umgerechnet
    gesamtstrecke_mm_post = proband_data_post['displacement_deg_or_mm'].abs().sum()
    gesamtstrecke_m_post = gesamtstrecke_mm_post / 1000  # Auf Meter umgerechnet
    
    # calculation of time
    proband_data_pre['gesamtstrecke_m'] = proband_data_pre.index * (gesamtstrecke_m_pre / (len(proband_data_pre) - 1))
    proband_data_post['gesamtstrecke_m'] = proband_data_post.index * (gesamtstrecke_m_post / (len(proband_data_post) - 1))

    # calculation of torque values
    proband_data_pre['torque_nm_abs'] = proband_data_pre['torque_nm'].abs()
    proband_data_post['torque_nm_abs'] = proband_data_post['torque_nm'].abs()

    # calculation of work
    work_done_pre = trapz(proband_data_pre['torque_nm_abs'], proband_data_pre['gesamtstrecke_m'])
    work_done_post = trapz(proband_data_post['torque_nm_abs'], proband_data_post['gesamtstrecke_m'])

    # add results
    total_work_done = pd.concat([
        total_work_done,
        pd.DataFrame({
            'id': [proband_id, proband_id],
            'experiment': ['pretest', 'posttest'],
            'group': [group, group],
            'work_done': [work_done_pre, work_done_post]
        })
    ], ignore_index=True)



C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\20225016.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [34]:
total_work_done.head()

,id,experiment,group,work_done
0,15073,pretest,beta_ala,1386.936605
1,15073,posttest,beta_ala,1598.201652
2,20438,pretest,beta_ala,1042.867295
3,20438,posttest,beta_ala,1931.843694
4,25778,pretest,beta_ala,1196.424275


# Calculate worf from Warmup to termination of protocol

In [36]:
# reset index
iso_reset = iso.reset_index(drop=True)

# filter conditions
pretest_conditions = (
    # pretest
    (iso_reset['id'] == 20438) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 24) |
    (iso_reset['id'] == 27351) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 39337) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 44858) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 49444) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 50843) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 24) |
    (iso_reset['id'] == 54145) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 54658) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 58046) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 59289) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 20) |
    (iso_reset['id'] == 61104) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 71620) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 73162) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 20) |
    (iso_reset['id'] == 76847) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 22)
)

posttest_conditions = (
    # posttest
    (iso_reset['id'] == 15073) & (iso_reset['measurement'] == 's6') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 20438) & (iso_reset['measurement'] == 's7') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 25778) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 26107) & (iso_reset['measurement'] == 's6') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 27351) & (iso_reset['measurement'] == 's1') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 28514) & (iso_reset['measurement'] == 's7') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 18) |
    (iso_reset['id'] == 33468) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 39750) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 24) |
    (iso_reset['id'] == 44858) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 49830) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 50843) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 54145) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 59289) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 16) |
    (iso_reset['id'] == 61104) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 76847) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 93302) & (iso_reset['measurement'] == 's1') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 93672) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28)
)

# combined conditions
combined_conditions = pretest_conditions | posttest_conditions

# use conditions and filter frame
warmup_abbruch = iso_reset[~combined_conditions].reset_index(drop=True)


In [37]:
def calculate_work_done(df):
    # extraxt ids
    probanden_ids = df['id'].unique()

    # create dataframe for work
    total_work_done = pd.DataFrame(columns=['id', 'experiment', 'group', 'work_done'])

    for proband_id in probanden_ids:
        proband_data = df[df['id'] == proband_id]

        # group information
        group = proband_data['group'].iloc[0]
    
        # filter for pre and posttest
        proband_data_pre = proband_data[proband_data['experiment'] == 'pretest'].copy()
        proband_data_post = proband_data[proband_data['experiment'] == 'posttest'].copy()

        # adjust time
        proband_data_pre.loc[:, 'time_ms'] = proband_data_pre.index * 5 + 5
        proband_data_post.reset_index(drop=True, inplace=True)
        proband_data_post.loc[:, 'time_ms'] = proband_data_post.index * 5 + 5

        # position change pre
        proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['pos_deg_or_mm'].diff()
        proband_data_post['displacement_deg_or_mm'] = proband_data_post['pos_deg_or_mm'].diff()
        proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['displacement_deg_or_mm'].where(proband_data_pre['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
        proband_data_post['displacement_deg_or_mm'] = proband_data_post['displacement_deg_or_mm'].where(proband_data_post['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
        
        # total distance pre
        gesamtstrecke_mm_pre = proband_data_pre['displacement_deg_or_mm'].abs().sum()
        gesamtstrecke_m_pre = gesamtstrecke_mm_pre / 1000  # Auf Meter umgerechnet
        gesamtstrecke_mm_post = proband_data_post['displacement_deg_or_mm'].abs().sum()
        gesamtstrecke_m_post = gesamtstrecke_mm_post / 1000  # Auf Meter umgerechnet
        
        # time
        proband_data_pre['gesamtstrecke_m'] = proband_data_pre.index * (gesamtstrecke_m_pre / (len(proband_data_pre) - 1))
        proband_data_post['gesamtstrecke_m'] = proband_data_post.index * (gesamtstrecke_m_post / (len(proband_data_post) - 1))

        # torque values
        proband_data_pre['torque_nm_abs'] = proband_data_pre['torque_nm'].abs()
        proband_data_post['torque_nm_abs'] = proband_data_post['torque_nm'].abs()

        # calculate work
        work_done_pre = trapz(proband_data_pre['torque_nm_abs'], proband_data_pre['gesamtstrecke_m'])
        work_done_post = trapz(proband_data_post['torque_nm_abs'], proband_data_post['gesamtstrecke_m'])

        # add results to frame
        total_work_done = pd.concat([
            total_work_done,
            pd.DataFrame({
                'id': [proband_id, proband_id],
                'experiment': ['pretest', 'posttest'],
                'group': [group, group],
                'work_done': [work_done_pre, work_done_post]
            })
        ], ignore_index=True)

    return total_work_done

# use for warmup_abbruch
total_work_done_warmup_abbruch = calculate_work_done(warmup_abbruch)


C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\3163125762.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [38]:
total_work_done_warmup_abbruch.head()

,id,experiment,group,work_done
0,15073,pretest,beta_ala,1386.936605
1,15073,posttest,beta_ala,1582.154194
2,20438,pretest,beta_ala,1013.765596
3,20438,posttest,beta_ala,1914.849472
4,25778,pretest,beta_ala,1196.424275


In [39]:
work_pre = total_work_done_warmup_abbruch[total_work_done_warmup_abbruch['experiment']=='pretest'].reset_index(drop=True)
work_post = total_work_done_warmup_abbruch[total_work_done_warmup_abbruch['experiment']=='posttest'].reset_index(drop=True)

In [40]:
work_pre['work_diff'] = work_post['work_done'] - work_pre['work_done']
work_pre

,id,experiment,group,work_done,work_diff
0,15073,pretest,beta_ala,1386.936605,195.217589
1,20438,pretest,beta_ala,1013.765596,901.083876
2,25778,pretest,beta_ala,1196.424275,-457.670353
3,26107,pretest,beta_ala,1902.483669,-339.155712
4,27351,pretest,beta_ala,487.111735,-113.342275
5,28514,pretest,placebo,1018.591019,14.309825
6,93672,pretest,beta_ala,909.051376,-6.835908
7,33468,pretest,placebo,694.327348,-137.898801
8,39337,pretest,placebo,1102.417785,204.866518
9,39750,pretest,placebo,584.580110,207.182604


In [43]:
(
    ggplot(work_pre, aes(x='group', y='work_diff', color='group'))
    + geom_boxplot()
    + labs(title='Pre to Post Work difference', subtitle='Set one to termination')
    + ylab('Work Difference [J]')
)

In [44]:
x = work_pre.query('group == "placebo"')['work_diff']
y = work_pre.query('group == "beta_ala"')['work_diff']

In [45]:
work_stat = pg.ttest(x, y, paired=False, alternative='less', correction='auto')
work_stat

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.320611,22,less,0.375765,"[-inf, 178.86]",0.130889,0.775,0.091104


In [188]:
total_work_done_warmup_abbruch.to_

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          48 non-null     object 
 1   experiment  48 non-null     object 
 2   group       48 non-null     object 
 3   work_done   48 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.6+ KB


In [56]:
(
    ggplot(total_work_done_warmup_abbruch, aes(x='experiment', y='work_done'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Total Work Done: Warm-up to termination')
    + ylab('Work [J]')
    + xlab('measurement')
)

In [47]:
work_wu_abb_stat = pg.mixed_anova(dv='work_done',between='group', within='experiment', subject='id', data=total_work_done_warmup_abbruch, correction='auto', effsize='np2')
work_wu_abb_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,844446.702052,1,22,844446.702052,4.103884,0.055082,0.157214,NaN
1,experiment,289.603811,1,22,289.603811,0.005885,0.939543,0.000267,1.0
2,Interaction,5058.093390,1,22,5058.093390,0.102792,0.751530,0.004651,NaN


In [48]:
posthoc_work_wu_abb = pg.pairwise_tests(data=total_work_done_warmup_abbruch, dv='work_done', between='group', subject='id', parametric=True, padjust='bonf', effsize='cohen')
posthoc_work_wu_abb

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,group,beta_ala,placebo,False,True,2.630256,46.0,two-sided,0.011567,4.342,0.75929


## Supplemental Plots and calculations for work done by time point

In [49]:
def calculate_work_done_per_measurement(df):
    # Eindeutige Probanden-IDs extrahieren
    probanden_ids = df['id'].unique()

    # DataFrame für die gesamte Arbeit erstellen
    total_work_done = pd.DataFrame(columns=['id', 'experiment', 'group', 'measurement', 'work_done'])

    for proband_id in probanden_ids:
        proband_data = df[df['id'] == proband_id]

        # Gruppeninfo abgreifen
        group = proband_data['group'].iloc[0]

        # Eindeutige Messzeitpunkte extrahieren
        measurements = proband_data['measurement'].unique()

        for measurement in measurements:
            measurement_data = proband_data[proband_data['measurement'] == measurement].copy()

            # Zeit-Spalte anpassen für den Messzeitpunkt
            measurement_data.loc[:, 'time_ms'] = measurement_data.index * 5 + 5

            # Index zurücksetzen, um wieder bei 5ms zu beginnen
            measurement_data.reset_index(drop=True, inplace=True)
            measurement_data.loc[:, 'time_ms'] = measurement_data.index * 5 + 5

            # Berechnung der Positionsänderung
            measurement_data['displacement_deg_or_mm'] = measurement_data['pos_deg_or_mm'].diff()

            # Filtere Werte über 0.7 aus der Spalte
            measurement_data['displacement_deg_or_mm'] = measurement_data['displacement_deg_or_mm'].where(
                measurement_data['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan
            )

            # Summierte Positionsänderungen als Gesamtstrecke
            gesamtstrecke_mm = measurement_data['displacement_deg_or_mm'].abs().sum()
            gesamtstrecke_m = gesamtstrecke_mm / 1000  # Auf Meter umgerechnet

            # Berechnung der normierten Zeit
            measurement_data['gesamtstrecke_m'] = measurement_data.index * (gesamtstrecke_m / (len(measurement_data) - 1))

            # Berechnung absoluter Torque Werte, um Drehmoment in jede Richtung zu erfassen
            measurement_data['torque_nm_abs'] = measurement_data['torque_nm'].abs()

            # Berechnung der Arbeit
            work_done = trapz(measurement_data['torque_nm_abs'], measurement_data['gesamtstrecke_m'])

            # Ergebnisse zum total_work_done DataFrame hinzufügen
            total_work_done = pd.concat([
                total_work_done,
                pd.DataFrame({
                    'id': [proband_id],
                    'experiment': [measurement_data['experiment'].iloc[0]],
                    'group': [group],
                    'measurement': [measurement],
                    'work_done': [work_done]
                })
            ], ignore_index=True)

    return total_work_done

# Anwenden auf warmup_abbruch
total_work_done_per_measurement = calculate_work_done_per_measurement(warmup_abbruch)


C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\680265839.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [50]:
total_work_done_posttest = calculate_work_done_per_measurement(warmup_abbruch[warmup_abbruch['experiment'] == 'posttest'])

C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\680265839.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [51]:
total_work_done_pretest = total_work_done_per_measurement.query('experiment == "pretest"')

In [52]:

grouped_means_pre = total_work_done_pretest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'mean'}).reset_index()
grouped_std_pre = total_work_done_pretest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'std'}).reset_index()

grouped_means_post = total_work_done_posttest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'mean'}).reset_index()
grouped_std_post = total_work_done_posttest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'std'}).reset_index()

# rename columns
grouped_means_pre.rename(columns={
    'work_done': 'work_done_mean',
}, inplace=True)

grouped_means_post.rename(columns={
    'work_done': 'work_done_mean',
}, inplace=True)

grouped_std_pre.rename(columns={
    'work_done': 'work_done_std',
}, inplace=True)

grouped_std_post.rename(columns={
    'work_done': 'work_done_std',
}, inplace=True)

In [53]:
grouped_means_pre = pd.concat([grouped_means_pre, grouped_std_pre.drop(['group', 'measurement'], axis=1)], axis=1)
grouped_means_post = pd.concat([grouped_means_post, grouped_std_post.drop(['group', 'measurement'], axis=1)], axis=1)

In [58]:
count_data = total_work_done_pretest.groupby(['group', 'measurement'])['work_done'].count().reset_index(name='count')

grouped_means_pre_with_count = pd.merge(grouped_means_pre, count_data, on=['group', 'measurement'])

# Plot erstellen
(
    ggplot(grouped_means_pre_with_count, aes(x='measurement', y='work_done_mean', color='group', group='group'))
    + geom_point(position=position_dodge(width=0.4))
    + geom_line(position=position_dodge(width=0.4))
    + geom_errorbar(
        aes(
            ymin=grouped_means_pre_with_count['work_done_mean'] - grouped_means_pre_with_count['work_done_std'],
            ymax=grouped_means_pre_with_count['work_done_mean'] + grouped_means_pre_with_count['work_done_std'],
            fill='group'
        ),
        position=position_dodge(width=0.4),
        linetype="blank"
    )
    + geom_text(
        aes(label='count'),
        position=position_dodge(width=0.4),
        nudge_x=0.4,
        vjust=-0.5,
        size=8
    )
    + labs(
        title='Pre-Test: means and std of work by group',
        subtitle='Number represents the amount of participants in the respective set'
    )
    + ylab('Work Done [J]')
    + xlab('Measurement')
    + scale_x_discrete(limits=['warmup', 'pt', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8'])
)


In [60]:
count_data = total_work_done_posttest.groupby(['group', 'measurement'])['work_done'].count().reset_index(name='count')
grouped_means_post_with_count = pd.merge(grouped_means_post, count_data, on=['group', 'measurement'])


(
    ggplot(grouped_means_post_with_count, aes(x='measurement', y='work_done_mean', color='group', group='group'))
    + geom_point(position=position_dodge(width=0.4))
    + geom_line(position=position_dodge(width=0.4))
    + geom_errorbar(
        aes(
            ymin=grouped_means_post_with_count['work_done_mean'] - grouped_means_post_with_count['work_done_std'],
            ymax=grouped_means_post_with_count['work_done_mean'] + grouped_means_post_with_count['work_done_std'],
            fill='group'
        ),
        position=position_dodge(width=0.4),
        linetype="blank"
    )
    + geom_text(
        aes(label='count'),
        position=position_dodge(width=0.4),
        nudge_x=0.4,
        vjust=-0.5,
        size=8
    )
    + labs(
        title='Post-Test: Means and std of work done bx group',
        subtitle='number represents the amount of participants in the respective set'
    )
    + ylab('Work Done [J]')
    + xlab('Measurement')
    + scale_x_discrete(limits=['warmup', 'pt', 's1', 's2', 's3', 's4', 's5', 's6', 's7'])
)


In [62]:
my_colors = ['#FF5733', '#33FF57', '#3366FF', '#FFFF33', '#33FFFF', '#FF33FF', '#33CC00',
             '#FF33CC', '#FF6633', '#33FF00', '#6600CC', '#33CC33', '#FF00FF', '#6600FF',
             '#00CCFF', '#CC33CC', '#0066FF', '#33CC99', '#66CC00', '#996633', '#00FF33',
             '#CC0033', '#33CC66', '#CC66FF']

(
    ggplot(total_work_done_per_measurement.query('experiment == "pretest"'), aes(x='measurement', y='work_done', color='id', group='id'))
    + geom_point()
    + geom_line()
    + scale_color_manual(values=my_colors)
    + labs(title='Individual work done in Pre-Test', subtitle = 'color = id')
    + theme(legend_position='none')
    + ylab('Work Done [J]')
    + xlab('measurement')
)

In [63]:
my_colors = ['#FF5733', '#33FF57', '#3366FF', '#FFFF33', '#33FFFF', '#FF33FF', '#33CC00',
             '#FF33CC', '#FF6633', '#33FF00', '#6600CC', '#33CC33', '#FF00FF', '#6600FF',
             '#00CCFF', '#CC33CC', '#0066FF', '#33CC99', '#66CC00', '#996633', '#00FF33',
             '#CC0033', '#33CC66', '#CC66FF']

(
    ggplot(total_work_done_posttest, aes(x='measurement', y='work_done', color='id', group='id'))
    + geom_point()
    + geom_line()
    + scale_color_manual(values=my_colors)
    + labs(title='Individual work done in Post-Test', subtitle = 'color = id')
    + theme(legend_position='none')
    + ylab('Work Done [J]')
    + xlab('measurement')
)

In [64]:
# number of participants in each set
participants_count_per_measurement = total_work_done_per_measurement.query('experiment == "pretest"').groupby('measurement')['id'].nunique().reset_index()

participants_count_per_measurement.columns = ['measurement', 'num_participants']

print(participants_count_per_measurement)

  measurement  num_participants
0          pt                24
1          s1                24
2          s2                24
3          s3                16
4          s4                 8
5          s5                 4
6          s6                 3
7          s7                 2
8          s8                 1
9      warmup                24


## Supplemental Material. Work done from peak torque to termination.

In [65]:
total_work_done_filtered = warmup_abbruch[warmup_abbruch['measurement'] != 'warmup'].reset_index(drop=True)

total_work_done_pt_abbruch = calculate_work_done(total_work_done_filtered)

C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\3163125762.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [67]:
(
    ggplot(total_work_done_pt_abbruch, aes(x='experiment', y='work_done'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Total Work Done: Peak Torque Test to termination')
    + ylab('Work in Joule')
    + xlab('Measurement')
)

In [68]:
work_pt_abb_stat = pg.mixed_anova(dv='work_done',between='group', within='experiment', subject='id', data=total_work_done_pt_abbruch, correction='auto', effsize='np2')
work_pt_abb_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,766708.186452,1,22,766708.186452,4.988162,0.036011,0.184828,NaN
1,experiment,12.960312,1,22,12.960312,0.000256,0.987370,0.000012,1.0
2,Interaction,7790.756492,1,22,7790.756492,0.154103,0.698422,0.006956,NaN


In [69]:
posthoc_pt_abb_stat = pg.pairwise_tests(data=total_work_done_pt_abbruch, dv='work_done', between='group', subject='id', parametric=True, padjust='bonf', effsize='cohen')
posthoc_pt_abb_stat

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,group,beta_ala,placebo,False,True,2.799066,46.0,two-sided,0.007465,6.113,0.808021


### Work done from fatigue set one to termination of exercise.

In [70]:
# filter warmup and peak torque
s1_abbruch = warmup_abbruch[(warmup_abbruch['measurement'] != 'warmup') & (warmup_abbruch['measurement'] != 'pt')].reset_index(drop=True)

# use function
total_work_done_s1_abbruch = calculate_work_done(s1_abbruch)

C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\3163125762.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [73]:
(
    ggplot(total_work_done_s1_abbruch, aes(x='experiment', y='work_done'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Total Work Done: set one to termination')
    + ylab('Work in Joule')
    + xlab('Measurement')
)

In [74]:
work_s1_abb_stat = pg.mixed_anova(dv='work_done',between='group', within='experiment', subject='id', data=total_work_done_s1_abbruch, correction='auto', effsize='np2')
work_s1_abb_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,628394.706537,1,22,628394.706537,4.297731,0.050079,0.163426,NaN
1,experiment,252.883635,1,22,252.883635,0.005241,0.942940,0.000238,1.0
2,Interaction,13794.991770,1,22,13794.991770,0.285916,0.598213,0.012829,NaN


In [75]:
posthoc_work_s1_abb = pg.pairwise_tests(data=total_work_done_s1_abbruch, dv='work_done', between='group', subject='id', parametric=True, padjust='bonf', effsize='cohen')
posthoc_work_s1_abb

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,group,beta_ala,placebo,False,True,2.595091,46.0,two-sided,0.012648,4.052,0.749138


# Determine fatigue index from set one to termination

In [76]:
s1_abbruch

,id,measurement,time_ms,pos_deg_or_mm,torque_nm,speed_deg/s_or_mm/s,torque_without_comp,rep,set,torque_dyn,force_right,force_left,experiment,group
0,15073,s1,5,11.4,21.8,0.0,72,1,1,-218,0,0,pretest,beta_ala
1,15073,s1,10,11.6,21.8,0.3,68,1,1,-218,0,0,pretest,beta_ala
2,15073,s1,15,11.6,22.5,1.8,65,1,1,-225,0,0,pretest,beta_ala
3,15073,s1,20,11.4,24.0,4.3,57,1,1,-240,0,0,pretest,beta_ala
4,15073,s1,25,11.4,26.7,5.8,53,1,1,-267,0,0,pretest,beta_ala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473081,93672,s2,43210,11.3,15.5,-15.3,150,28,1,-155,0,0,posttest,beta_ala
1473082,93672,s2,43215,11.1,20.3,-10.0,102,28,1,-203,0,0,posttest,beta_ala
1473083,93672,s2,43220,11.1,25.2,-8.3,53,28,1,-252,0,0,posttest,beta_ala
1473084,93672,s2,43225,11.1,30.0,-8.0,5,28,1,-300,0,0,posttest,beta_ala


## mean of top thee torque peaks

In [77]:
peaks_largest = []

for _, sub_df in iso.groupby(['id', 'experiment', 'group']):
    proband_id, experiment_type, group = _
    
    px = sub_df.query('rep % 2 == 0').copy()
    
    px['torque_nm_abs'] = px['torque_nm'].abs()
    px.loc[:, 'time_ms'] = px.index * 5 + 5
    peaks, properties = find_peaks(px['torque_nm_abs'], height=50, distance=200)
    top_three_peaks = px['torque_nm_abs'].iloc[peaks].nlargest(3)
    mean_top_three = top_three_peaks.mean()
    
    
    # save results
    peaks_largest.append({
        'id': proband_id,
        'experiment': experiment_type,
        'group': group,
        'top_three_peaks': top_three_peaks.tolist(),
        'mean_top_three': mean_top_three,
    })

# to dataframe
peaks_largest = pd.DataFrame(peaks_largest)



C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\3135058859.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, sub_df in iso.groupby(['id', 'experiment', 'group']):


## mean of peaks from last three reps

In [78]:
means_last_reps = pd.read_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Zusammenfassung Arbeitssätze\Absolvierte_reps.xlsx", sheet_name='Sheet2')

In [79]:
peaks_largest = pd.merge(peaks_largest, means_last_reps[['experiment', 'id', 'group', 'Mean_peaks']], 
                         on=['experiment', 'id', 'group'], how='left')



### calculate fatigue index

In [80]:
peaks_largest['fatigue_index'] = ((peaks_largest['mean_top_three'] - peaks_largest['Mean_peaks']) / 
                                peaks_largest['mean_top_three']) * 100
peaks_largest

,id,experiment,group,top_three_peaks,mean_top_three,Mean_peaks,fatigue_index
0,15073,posttest,beta_ala,"[269.0, 264.8, 249.0]",260.933333,121.666667,53.372509
1,15073,pretest,beta_ala,"[249.5, 248.0, 243.0]",246.833333,123.033333,50.155300
2,20438,posttest,beta_ala,"[291.0, 284.7, 284.0]",286.566667,128.433333,55.182040
3,20438,pretest,beta_ala,"[306.8, 294.0, 292.2]",297.666667,145.566667,51.097424
4,25778,posttest,beta_ala,"[155.7, 152.3, 147.5]",151.833333,67.166667,55.762898
5,25778,pretest,beta_ala,"[163.2, 154.5, 145.5]",154.400000,77.833333,49.589810
6,26107,posttest,beta_ala,"[195.0, 194.0, 186.5]",191.833333,92.166667,51.954822
7,26107,pretest,beta_ala,"[201.5, 186.8, 182.0]",190.100000,94.166667,50.464668
8,27351,posttest,beta_ala,"[166.5, 161.0, 156.0]",161.166667,70.200000,56.442606
9,27351,pretest,beta_ala,"[162.8, 162.8, 155.0]",160.200000,68.033333,57.532251


## pre to post difference of fatigue index

In [81]:
fatigue_pre = peaks_largest[peaks_largest['experiment'] == 'pretest'].reset_index(drop=True)
fatigue_post = peaks_largest[peaks_largest['experiment'] == 'posttest'].reset_index(drop=True)

In [82]:
fatigue_pre['diff_fatigue_index'] = fatigue_post['fatigue_index'] - fatigue_pre['fatigue_index']
fatigue_pre

,id,experiment,group,top_three_peaks,mean_top_three,Mean_peaks,fatigue_index,diff_fatigue_index
0,15073,pretest,beta_ala,"[249.5, 248.0, 243.0]",246.833333,123.033333,50.155300,3.217208
1,20438,pretest,beta_ala,"[306.8, 294.0, 292.2]",297.666667,145.566667,51.097424,4.084616
2,25778,pretest,beta_ala,"[163.2, 154.5, 145.5]",154.400000,77.833333,49.589810,6.173088
3,26107,pretest,beta_ala,"[201.5, 186.8, 182.0]",190.100000,94.166667,50.464668,1.490154
4,27351,pretest,beta_ala,"[162.8, 162.8, 155.0]",160.200000,68.033333,57.532251,-1.089645
5,28514,pretest,placebo,"[121.2, 119.0, 118.2]",119.466667,62.433333,47.739955,8.179760
6,33468,pretest,placebo,"[158.0, 157.5, 156.5]",157.333333,72.533333,53.898305,-2.678293
7,39337,pretest,placebo,"[296.7, 274.2, 268.5]",279.800000,145.700000,47.927091,7.408058
8,39750,pretest,placebo,"[139.2, 135.5, 131.7]",135.466667,65.700000,51.500984,-0.061396
9,44858,pretest,beta_ala,"[143.3, 142.2, 140.7]",142.066667,65.766667,53.707180,-3.606124


In [83]:
(
    ggplot(fatigue_pre, aes(x='group', y='diff_fatigue_index', color='group'))
    + geom_boxplot()
    + ylab('Difference of Fatigue Index')
)

In [84]:
x = fatigue_pre.query('group == "placebo"')['diff_fatigue_index']
y = fatigue_pre.query('group == "beta_ala"')['diff_fatigue_index']

work_stat = pg.ttest(x, y, paired=False, alternative='less', correction='auto')
work_stat

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.072912,22,less,0.85253,"[-inf, 5.67]",0.438014,0.884,0.003619


# TMG Analysis

In [86]:
tmg_raw = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/tmg_raw_data.csv?token=BGUXDRHGN5QCL7MI2EWSL7DF4X5OI")
tmg_raw.rename(columns={'Unnamed: 0': 'index', 'value': 'displacement'}, inplace=True)

# adjust datatypes
tmg_raw = tmg_raw.astype({'group': 'string', 'experiment': 'string', 'measurement': 'string'})

# add time
millisekenden_pro_datenpunkt = 1
tmg_raw['time_ms'] = tmg_raw.groupby(['id', 'experiment', 'measurement'])['index'].transform(lambda x: (x - x.min()) * millisekenden_pro_datenpunkt)

tmg_raw

,index,id,group,experiment,measurement,displacement,time_ms
0,0,15073,beta_ala,posttest,supine_1,0.000000,0
1,1,15073,beta_ala,posttest,supine_1,-0.000108,1
2,2,15073,beta_ala,posttest,supine_1,-0.000600,2
3,3,15073,beta_ala,posttest,supine_1,-0.001181,3
4,4,15073,beta_ala,posttest,supine_1,-0.001808,4
...,...,...,...,...,...,...,...
500995,501995,93672,beta_ala,pretest,supine_4,0.008989,995
500996,501996,93672,beta_ala,pretest,supine_4,0.004278,996
500997,501997,93672,beta_ala,pretest,supine_4,0.001684,997
500998,501998,93672,beta_ala,pretest,supine_4,0.000508,998


In [10]:
example_data = tmg_raw.query('id == 25778 and experiment == "pretest" and measurement == "s4"')

# threshold for peaks Peaks
threshold = 0.04

# Wert des ersten Peaks der Kurve für dieses spezifische Beispiel
peaks, _ = find_peaks(example_data['displacement'].values, height=threshold)
dm = example_data['displacement'].iloc[peaks[0]] if len(peaks) > 0 else np.nan
time_of_dm = example_data['time_ms'].iloc[peaks[0]] if len(peaks) > 0 else np.nan

# Td berechnen: Schwellenwert für Td
threshold_td = 0.1 * dm
# Index von Td berechnen
index_td = example_data.index[example_data['displacement'] >= threshold_td].min()
# Zeit für Td berechnen
time_td = example_data.loc[index_td, 'time_ms'] if pd.notna(index_td) else np.nan

# Schwellenwert für Tc
threshold_tc = 0.9 * dm
# Index von Td berechnen
index_tc = example_data.index[example_data['displacement'] >= threshold_tc].min()
# Zeit für Td berechnen
time_tc = example_data.loc[index_tc, 'time_ms'] if pd.notna(index_tc) else np.nan


# Finde den ersten Wert, der 50% von dm entspricht
threshold_ts = 0.5 * dm
erster_wert_index = None

for i, wert in enumerate(example_data['displacement'].values):
    if wert >= threshold_ts:
        erster_wert_index = i
        break

# Iteriere weiter ab diesem Wert und finde den nächsten Wert, der 50% von dm entspricht
zweiter_wert_index = None

if erster_wert_index is not None:
    for y, wert in enumerate(example_data['displacement'].values[erster_wert_index + 1:]):
        if wert <= threshold_ts:
            zweiter_wert_index = erster_wert_index + 1 + y
            break

# Überprüfe, ob die Indizes gültig sind
if erster_wert_index is not None and zweiter_wert_index is not None:
    erster_wert_index += example_data.index[0]
    zweiter_wert_index += example_data.index[0]

# Berechne die Zeiten für ts_start und ts_end
time_ts_start = example_data.loc[erster_wert_index, 'time_ms'] if erster_wert_index is not None else np.nan
time_ts_end = example_data.loc[zweiter_wert_index, 'time_ms'] if zweiter_wert_index is not None else np.nan

# Finde den ersten Wert, der 90% von dm entspricht
threshold_tr = 0.9 * dm
erste_zahl_index = None

for i, zahl in enumerate(example_data['displacement'].values):
    if zahl >= threshold_tr:
        erste_zahl_index = i
        break

# Iteriere weiter ab diesem Wert und finde den nächsten Wert, der 90% von dm entspricht
zweite_zahl_index = None

if erste_zahl_index is not None:
    for y, zahl in enumerate(example_data['displacement'].values[erste_zahl_index + 1:]):
        if zahl <= threshold_tr:
            zweite_zahl_index = erste_zahl_index + 1 + y
            break

# Überprüfe, ob die Indizes gültig sind
if zweite_zahl_index is not None and zweite_zahl_index is not None:
    erste_zahl_index += example_data.index[0]
    zweite_zahl_index += example_data.index[0]
    
index_tr_end = example_data.index[example_data['time_ms'] == time_ts_end].min()
    
time_tr_start = example_data.loc[zweite_zahl_index, 'time_ms'] if zweite_zahl_index is not None else np.nan
time_tr_end = example_data.loc[index_tr_end, 'time_ms'] if index_tr_end is not None else np.nan

print("Dm:", dm, 'mm')
print("Td:", time_td, 'ms')
print("Tc:", time_tc, 'ms')
print("Ts:", time_ts_end - time_ts_start, 'ms')
print('Tr:', time_tr_end - time_tr_start, 'ms')
print('Vc:', (dm / (time_td + time_tc)), 'mm/ms')


# Deine ggplot-Grafik bleibt unverändert, füge einfach die Punkte für ts_start und ts_end hinzu
(
    ggplot(example_data, aes(x='time_ms', y='displacement'))
    + geom_line()
    + geom_point(data=example_data.iloc[[peaks[0]]], mapping=aes(x='time_ms', y='displacement'), color='red', size=3)
    + geom_point(aes(x='time_tc', y='displacement'), data=pd.DataFrame({'time_tc': [time_tc], 'displacement': [dm * 0.9]}), color='green', size=3)
    + geom_point(aes(x='time_td', y='displacement'), data=pd.DataFrame({'time_td': [time_td], 'displacement': [dm * 0.1]}), color='blue', size=3)
    + geom_point(aes(x='time_ts_start', y='displacement'), data=pd.DataFrame({'time_ts_start': [time_ts_start], 'displacement': [dm * 0.5]}), color='orange', size=3)
    + geom_point(aes(x='time_ts_end', y='displacement'), data=pd.DataFrame({'time_ts_end': [time_ts_end], 'displacement': [dm * 0.5]}), color='orange', size=3)
    + geom_point(aes(x='time_tr_start', y='displacement'), data=pd.DataFrame({'time_tr_start': [time_tr_start], 'displacement': [dm * 0.9]}), color='black', size=3)
    + geom_point(aes(x='time_tr_end', y='displacement'), data=pd.DataFrame({'time_tr_end': [time_tr_end], 'displacement': [dm * 0.9]}), color='black', size=3)
)


Dm: 0.415878055549873 mm
Td: 8 ms
Tc: 29 ms
Ts: 22 ms
Tr: 5 ms
Vc: 0.011239947447293865 mm/ms


# Calculte individual tmg parameters for each measurement

In [87]:
# create dataframe
tmg_parameter = pd.DataFrame(columns=['id', 'group', 'experiment', 'measurement', 'Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc'])

# threshold for Peaks
threshold = 0.04

# iterate by ID
for unique_id in tmg_raw['id'].unique():
    # for pre and post test
    for unique_experiment in tmg_raw['experiment'].unique():
        # measurement time point
        for unique_measurement in tmg_raw['measurement'].unique():
            # filter data
            subset = tmg_raw[(tmg_raw['id'] == unique_id) & 
                             (tmg_raw['experiment'] == unique_experiment) & 
                             (tmg_raw['measurement'] == unique_measurement)].copy()
            
            if not subset.empty:
                parameter_value = subset['displacement'].mean()
                
                # group
                group = subset['group'].iloc[0]

                # value of first peak
                peaks, _ = find_peaks(subset['displacement'].values, height=threshold)
                dm = subset['displacement'].iloc[peaks[0]] if len(peaks) > 0 else np.nan

                # Td threshold
                threshold_td = 0.1 * dm
                index_td = subset.index[subset['displacement'] >= threshold_td].min()
                # Td time
                time_td = subset.loc[index_td, 'time_ms'] if pd.notna(index_td) and pd.notna(subset.loc[index_td, 'time_ms']) else np.nan

                # threshold Tc
                threshold_tc = 0.9 * dm
                index_tc = subset.index[subset['displacement'] >= threshold_tc].min()
                # time tc
                time_tc = subset.loc[index_tc, 'time_ms'] if pd.notna(index_tc) and pd.notna(subset.loc[index_tc, 'time_ms']) else np.nan

                # 50% dm
                threshold_ts = 0.5 * dm
                erster_wert_index = None

                for i, wert in enumerate(subset['displacement'].values):
                    if wert >= threshold_ts:
                        erster_wert_index = i
                        break

                # next 50% dm
                zweiter_wert_index = None

                if erster_wert_index is not None:
                    for y, wert in enumerate(subset['displacement'].values[erster_wert_index + 1:]):
                        if wert <= threshold_ts:
                            zweiter_wert_index = erster_wert_index + 1 + y
                            break

                if erster_wert_index is not None and zweiter_wert_index is not None:
                    erster_wert_index += subset.index[0]
                    zweiter_wert_index += subset.index[0]

                # time ts start and end
                time_ts_start = subset.loc[erster_wert_index, 'time_ms'] if pd.notna(erster_wert_index) and pd.notna(subset.loc[erster_wert_index, 'time_ms']) else np.nan
                time_ts_end = subset.loc[zweiter_wert_index, 'time_ms'] if pd.notna(zweiter_wert_index) and pd.notna(subset.loc[zweiter_wert_index, 'time_ms']) else np.nan

                # 90% dm
                threshold_tr = 0.9 * dm
                erste_zahl_index = None

                for i, zahl in enumerate(subset['displacement'].values):
                    if zahl >= threshold_tr:
                        erste_zahl_index = i
                        break

                # next 90% dm
                zweite_zahl_index = None

                if erste_zahl_index is not None:
                    for y, zahl in enumerate(subset['displacement'].values[erste_zahl_index + 1:]):
                        if zahl <= threshold_tr:
                            zweite_zahl_index = erste_zahl_index + 1 + y
                            break

                if zweite_zahl_index is not None and zweite_zahl_index is not None:
                    erste_zahl_index += subset.index[0]
                    zweite_zahl_index += subset.index[0]

                index_tr_end = subset.index[subset['time_ms'] == time_ts_end].min()

                # time tr start and end
                time_tr_start = subset.loc[zweite_zahl_index, 'time_ms'] if pd.notna(zweite_zahl_index) and pd.notna(subset.loc[zweite_zahl_index, 'time_ms']) else np.nan
                time_tr_end = subset.loc[index_tr_end, 'time_ms'] if pd.notna(index_tr_end) and pd.notna(subset.loc[index_tr_end, 'time_ms']) else np.nan

                tmg_parameter = pd.concat([tmg_parameter, pd.DataFrame({
                    'id': [unique_id],
                    'group':[group],
                    'experiment': [unique_experiment],
                    'measurement': [unique_measurement],
                    'Dm': [dm],
                    'Td': [time_td],
                    'Tc': [time_tc],
                    'Ts': [time_ts_end - time_ts_start],
                    'Tr': [time_tr_end - time_tr_start],
                    'Vc': [dm / (time_td + time_tc)]
                })], ignore_index=True)

tmg_parameter


C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\2704293734.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmg_parameter = pd.concat([tmg_parameter, pd.DataFrame({


,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1,9.949456,31,56,77,46,0.114362
1,15073,beta_ala,posttest,before_warmup,2.989571,23,49,179,146,0.041522
2,15073,beta_ala,posttest,after_warmup,2.272929,20,43,194,36,0.036078
3,15073,beta_ala,posttest,after_pt,1.812468,19,40,167,138,0.030720
4,15073,beta_ala,posttest,s1,0.742931,16,33,26,7,0.015162
...,...,...,...,...,...,...,...,...,...,...
496,93672,beta_ala,pretest,s1,3.199221,22,41,28,8,0.050781
497,93672,beta_ala,pretest,s2,2.277724,24,43,30,9,0.033996
498,93672,beta_ala,pretest,supine_2,8.410425,25,51,137,103,0.110663
499,93672,beta_ala,pretest,supine_3,11.591811,25,54,144,111,0.146732


In [88]:
tmg_parameter = tmg_parameter.astype({'group': 'category', 'experiment': 'category', 'measurement': 'category', 
                                       'Td': 'float64', 'Tc': 'float64', 'Dm': 'float64', 'Ts': 'float64', 'Tr': 'float64',
                                      'Vc': 'float64'})

tmg_parameter['experiment'] = pd.Categorical(tmg_parameter['experiment'], categories=['pretest', 'posttest'], ordered=True)

tmg_parameter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id           501 non-null    object  
 1   group        501 non-null    category
 2   experiment   501 non-null    category
 3   measurement  501 non-null    category
 4   Dm           501 non-null    float64 
 5   Td           501 non-null    float64 
 6   Tc           501 non-null    float64 
 7   Ts           501 non-null    float64 
 8   Tr           501 non-null    float64 
 9   Vc           501 non-null    float64 
dtypes: category(3), float64(6), object(1)
memory usage: 29.9+ KB


In [89]:
# adding pre or posttest
tmg_parameter['measurement'] = tmg_parameter.apply(lambda row: row['measurement']
                                                   + '_pre' if 'pretest' in row['experiment'] else row['measurement'], axis=1)
tmg_parameter['measurement'] = tmg_parameter.apply(lambda row: row['measurement']
                                                   + '_post' if 'posttest' in row['experiment'] else row['measurement'], axis=1)

In [90]:
tmg_parameter.head(-1)

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1_post,9.949456,31.0,56.0,77.0,46.0,0.114362
1,15073,beta_ala,posttest,before_warmup_post,2.989571,23.0,49.0,179.0,146.0,0.041522
2,15073,beta_ala,posttest,after_warmup_post,2.272929,20.0,43.0,194.0,36.0,0.036078
3,15073,beta_ala,posttest,after_pt_post,1.812468,19.0,40.0,167.0,138.0,0.030720
4,15073,beta_ala,posttest,s1_post,0.742931,16.0,33.0,26.0,7.0,0.015162
...,...,...,...,...,...,...,...,...,...,...
495,93672,beta_ala,pretest,after_pt_pre,5.181711,20.0,37.0,27.0,9.0,0.090907
496,93672,beta_ala,pretest,s1_pre,3.199221,22.0,41.0,28.0,8.0,0.050781
497,93672,beta_ala,pretest,s2_pre,2.277724,24.0,43.0,30.0,9.0,0.033996
498,93672,beta_ala,pretest,supine_2_pre,8.410425,25.0,51.0,137.0,103.0,0.110663


# Seated TMG Analysis. Differences from pre to post test

In [120]:
# to view data download the excel file tmg_parameter and copy its path to the brackets after pd.read_excel()
# keep sheet name part
diff_df = pd.read_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\TMG Rohdaten Frame\tmg_parameter.xlsx", sheet_name='tmg_diff')
diff_df = diff_df.astype({'group': 'category', 'experiment': 'category'})
diff_df = diff_df.sort_values(by='experiment', ascending=True)
diff_df

,id,group,experiment,Dm,Td,Tc,Ts,Tr,Vc
47,93672,beta_ala,posttest,-3.647600,2,5,3,1,-0.070517
25,20438,beta_ala,posttest,-1.236830,1,6,24,10,-0.026392
26,25778,beta_ala,posttest,-1.067054,1,4,2,0,-0.021753
27,26107,beta_ala,posttest,-0.706058,3,2,-202,-202,-0.013451
28,27351,beta_ala,posttest,-0.525296,0,0,21,26,-0.008081
29,28514,placebo,posttest,-1.429971,0,1,-69,-68,-0.028223
30,33468,placebo,posttest,0.393301,3,8,-58,29,-0.007598
31,39337,placebo,posttest,-1.792409,2,4,3,1,-0.034555
32,39750,placebo,posttest,-1.831496,0,-1,-3,-2,-0.035066
33,44858,beta_ala,posttest,-0.295869,6,10,-91,-99,-0.017450


## diff Dm

In [123]:
(
    ggplot(diff_df, aes(x='experiment', y='Dm', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Dm Difference [mm]')
    + xlab('Measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [124]:
dm_stat = pg.mixed_anova(dv='Dm',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
dm_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.090000,1,22,0.090000,0.043583,0.836551,0.001977,NaN
1,experiment,0.051341,1,22,0.051341,0.080825,0.778841,0.003660,1.0
2,Interaction,0.059653,1,22,0.059653,0.093911,0.762144,0.004251,NaN


## Diff Td

In [125]:
(
    ggplot(diff_df, aes(x='experiment', y='Td', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Td Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [126]:
Td_stat = pg.mixed_anova(dv='Td',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Td_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,2.520833,1,22,2.520833,0.270583,0.608136,0.012150,NaN
1,experiment,4.687500,1,22,4.687500,1.209086,0.283401,0.052095,1.0
2,Interaction,3.520833,1,22,3.520833,0.908158,0.350958,0.039643,NaN


## diff Tc

In [127]:
(
    ggplot(diff_df, aes(x='experiment', y='Tc', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Tc Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [128]:
Tc_stat = pg.mixed_anova(dv='Tc',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Tc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,17.520833,1,22,17.520833,0.357637,0.555928,0.015996,NaN
1,experiment,6.020833,1,22,6.020833,0.279768,0.602150,0.012557,1.0
2,Interaction,11.020833,1,22,11.020833,0.512101,0.481758,0.022748,NaN


## diff Ts

In [129]:
(
    ggplot(diff_df, aes(x='experiment', y='Ts', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Ts Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [130]:
Ts_stat = pg.mixed_anova(dv='Ts',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Ts_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,229.687500,1,22,229.687500,0.021952,0.883563,0.000997,NaN
1,experiment,1485.187500,1,22,1485.187500,0.947096,0.341039,0.041273,1.0
2,Interaction,105.020833,1,22,105.020833,0.066971,0.798205,0.003035,NaN


## diff Tr

In [131]:
(
    ggplot(diff_df, aes(x='experiment', y='Tr', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Tr Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [132]:
Tr_stat = pg.mixed_anova(dv='Tr',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Tr_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,660.083333,1,22,660.083333,0.062098,0.805523,0.002815,NaN
1,experiment,5808.000000,1,22,5808.000000,1.754691,0.198884,0.073867,1.0
2,Interaction,133.333333,1,22,133.333333,0.040282,0.842774,0.001828,NaN


## diff Vc

In [134]:
(
    ggplot(diff_df, aes(x='experiment', y='Vc', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Vc Difference [mm/ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [135]:
Vc_stat = pg.mixed_anova(dv='Vc',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Vc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,1.273159e-07,1,22,1.273159e-07,0.000251,0.987497,0.000011,NaN
1,experiment,3.343757e-05,1,22,3.343757e-05,0.253255,0.619794,0.011381,1.0
2,Interaction,2.605676e-05,1,22,2.605676e-05,0.197353,0.661205,0.008891,NaN


# Supine TMG Analysis

In [91]:
relevant_measurements = ['supine_1_pre', 'supine_2_pre', 'supine_3_pre', 'supine_4_pre',
                         'supine_1_post', 'supine_2_post', 'supine_3_post', 'supine_4_post']
filtered_data = tmg_parameter[tmg_parameter['measurement'].isin(relevant_measurements)]

filtered_data = filtered_data.sort_values(by=['id', 'group', 'experiment']).reset_index(drop=True)
pretest_data = filtered_data[filtered_data['experiment'] == 'pretest']
posttest_data = filtered_data[filtered_data['experiment'] == 'posttest']


In [92]:
pretest_data = pretest_data.reset_index(drop=True)
pretest_data

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,pretest,supine_1_pre,9.726150,31.0,57.0,106.0,72.0,0.110524
1,15073,beta_ala,pretest,supine_2_pre,5.774841,28.0,56.0,98.0,56.0,0.068748
2,15073,beta_ala,pretest,supine_3_pre,9.100387,30.0,57.0,149.0,110.0,0.104602
3,15073,beta_ala,pretest,supine_4_pre,9.112594,29.0,59.0,104.0,64.0,0.103552
4,20438,beta_ala,pretest,supine_1_pre,10.232495,26.0,50.0,44.0,19.0,0.134638
...,...,...,...,...,...,...,...,...,...,...
91,93302,placebo,pretest,supine_4_pre,7.584031,27.0,56.0,51.0,21.0,0.091374
92,93672,beta_ala,pretest,supine_1_pre,11.228515,26.0,53.0,172.0,140.0,0.142133
93,93672,beta_ala,pretest,supine_2_pre,8.410425,25.0,51.0,137.0,103.0,0.110663
94,93672,beta_ala,pretest,supine_3_pre,11.591811,25.0,54.0,144.0,111.0,0.146732


In [93]:
posttest_data = posttest_data.reset_index(drop=True)
posttest_data

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1_post,9.949456,31.0,56.0,77.0,46.0,0.114362
1,15073,beta_ala,posttest,supine_2_post,6.965459,28.0,57.0,86.0,46.0,0.081947
2,15073,beta_ala,posttest,supine_3_post,8.982938,30.0,57.0,140.0,104.0,0.103252
3,15073,beta_ala,posttest,supine_4_post,9.926545,30.0,56.0,124.0,88.0,0.115425
4,20438,beta_ala,posttest,supine_1_post,8.599606,27.0,52.0,49.0,21.0,0.108856
...,...,...,...,...,...,...,...,...,...,...
91,93302,placebo,posttest,supine_4_post,8.152583,25.0,53.0,49.0,19.0,0.104520
92,93672,beta_ala,posttest,supine_1_post,10.276126,25.0,49.0,173.0,142.0,0.138867
93,93672,beta_ala,posttest,supine_2_post,9.089581,25.0,50.0,153.0,123.0,0.121194
94,93672,beta_ala,posttest,supine_3_post,11.476495,26.0,54.0,136.0,105.0,0.143456


In [94]:
result_data = pd.DataFrame()

# Hinzufügen von 'group', 'measurement' und 'id'
result_data['group'] = pretest_data['group']
result_data['measurement'] = pretest_data['measurement']
result_data['id'] = pretest_data['id']

# Berechnung der Differenzen für alle Parameter
diff_columns = ['Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc']
for col in diff_columns:
    result_data[f'diff_{col}'] = posttest_data[col] - pretest_data[col]

# Anzeigen des Ergebnisses
result_data

,group,measurement,id,diff_Dm,diff_Td,diff_Tc,diff_Ts,diff_Tr,diff_Vc
0,beta_ala,supine_1_pre,15073,0.223306,0.0,-1.0,-29.0,-26.0,0.003837
1,beta_ala,supine_2_pre,15073,1.190617,0.0,1.0,-12.0,-10.0,0.013198
2,beta_ala,supine_3_pre,15073,-0.117449,0.0,0.0,-9.0,-6.0,-0.001350
3,beta_ala,supine_4_pre,15073,0.813951,1.0,-3.0,20.0,24.0,0.011873
4,beta_ala,supine_1_pre,20438,-1.632890,1.0,2.0,5.0,2.0,-0.025782
...,...,...,...,...,...,...,...,...,...
91,placebo,supine_4_pre,93302,0.568552,-2.0,-3.0,-2.0,-2.0,0.013146
92,beta_ala,supine_1_pre,93672,-0.952389,-1.0,-4.0,1.0,2.0,-0.003267
93,beta_ala,supine_2_pre,93672,0.679156,0.0,-1.0,16.0,20.0,0.010531
94,beta_ala,supine_3_pre,93672,-0.115316,1.0,0.0,-8.0,-6.0,-0.003276


In [95]:
mean_std_supine = result_data.groupby(['group', 'measurement']).agg({
    'diff_Dm': ['mean', 'std'],
    'diff_Td': ['mean', 'std'],
    'diff_Tc': ['mean', 'std'],
    'diff_Ts': ['mean', 'std'],
    'diff_Tr': ['mean', 'std'],
    'diff_Vc': ['mean', 'std'],    
})

mean_std_supine = mean_std_supine.reset_index()
mean_std_supine.columns = ['group', 'measurement',
                           'diff_Dm_mean', 'diff_Dm_std',
                           'diff_Td_mean', 'diff_Td_std',
                           'diff_Tc_mean', 'diff_Tc_std',
                           'diff_Ts_mean', 'diff_Ts_std',
                           'diff_Tr_mean', 'diff_Tr_std',
                           'diff_Vc_mean', 'diff_Vc_std']
mean_std_supine

C:\Users\bvenn\AppData\Local\Temp\ipykernel_1320\1810155329.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_std_supine = result_data.groupby(['group', 'measurement']).agg({


,group,measurement,diff_Dm_mean,diff_Dm_std,diff_Td_mean,diff_Td_std,diff_Tc_mean,diff_Tc_std,diff_Ts_mean,diff_Ts_std,diff_Tr_mean,diff_Tr_std,diff_Vc_mean,diff_Vc_std
0,beta_ala,supine_1_pre,-0.412466,1.231345,-0.083333,0.996205,-1.416667,3.396745,0.583333,25.903521,2.666667,29.370156,-0.002972,0.016618
1,beta_ala,supine_2_pre,0.282381,0.695844,0.333333,2.964436,1.166667,4.302924,5.750000,15.903830,3.750000,16.153384,0.002103,0.013790
2,beta_ala,supine_3_pre,-0.603422,1.364757,0.000000,2.044949,-1.166667,3.040136,6.083333,26.005099,4.916667,28.398010,-0.005608,0.019043
3,beta_ala,supine_4_pre,0.031945,0.772378,0.250000,0.866025,-1.416667,1.975225,-1.583333,48.299178,-1.166667,45.934406,0.001509,0.009516
4,placebo,supine_1_pre,-0.198761,1.967166,0.000000,1.809068,-1.583333,5.991787,-13.750000,41.209277,-9.333333,40.614224,0.000280,0.028788
5,placebo,supine_2_pre,0.355708,1.478547,-0.583333,1.443376,-1.416667,4.601548,-2.583333,23.157399,-1.416667,22.837204,0.006553,0.019645
6,placebo,supine_3_pre,-0.245030,1.698245,0.000000,1.595448,0.333333,3.284491,-19.500000,45.364183,-12.250000,49.900128,-0.002420,0.020470
7,placebo,supine_4_pre,-0.070708,1.736899,-0.333333,1.723281,0.333333,4.996969,-17.000000,45.889195,-15.750000,47.342322,-0.000465,0.023840


In [12]:
# extrahieren des data frames mit differenzen
#result_data.to_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Dataframe\differences_supine.xlsx")

In [96]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Td', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Td Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [97]:
# distribution
(
    ggplot(result_data, aes(x='diff_Td'))
    + geom_histogram()
    + ylab('Td Difference [ms]')
)

In [99]:
Td_stat = pg.mixed_anova(dv='diff_Td',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Td_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,3.010417,1,22,3.010417,0.621992,0.438725,0.027495,NaN
1,measurement,0.197917,3,66,0.065972,0.024890,0.994639,0.001130,0.916476
2,Interaction,4.114583,3,66,1.371528,0.517447,0.671719,0.022980,NaN


In [100]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Tc', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Tc Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [101]:
# distribution
(
    ggplot(result_data, aes(x='diff_Tc'))
    + geom_histogram()
    + ylab('Td Diff [ms]')
)

In [102]:
Tc_stat = pg.mixed_anova(dv='diff_Tc',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Tc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.375000,1,22,0.375000,0.015534,0.901943,0.000706,NaN
1,measurement,25.541667,3,66,8.513889,0.582146,0.628810,0.025779,0.886621
2,Interaction,71.708333,3,66,23.902778,1.634378,0.189841,0.069153,NaN


In [103]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Dm', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Dm Difference [mm]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [104]:
# distribution
(
    ggplot(result_data, aes(x='diff_Dm'))
    + geom_histogram()
    + ylab('Td Diff [ms]')
)

In [105]:
Dm_stat = pg.mixed_anova(dv='diff_Dm',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Dm_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.441899,1,22,0.441899,0.096011,0.759586,0.004345,NaN
1,measurement,7.902471,3,66,2.634157,2.191772,0.097258,0.090600,0.945033
2,Interaction,0.698274,3,66,0.232758,0.193668,0.900355,0.008726,NaN


In [106]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Ts', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Ts Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [107]:
# distribution
(
    ggplot(result_data, aes(x='diff_Ts'))
    + geom_histogram()
    + ylab('Ts Diff [ms]')
)

In [108]:
Ts_stat = pg.mixed_anova(dv='diff_Ts',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Ts_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,6080.166667,1,22,6080.166667,3.037713,0.095313,0.121325,NaN
1,measurement,1606.416667,3,66,535.472222,0.507711,0.678321,0.022557,0.82267
2,Interaction,922.250000,3,66,307.416667,0.291479,0.831402,0.013076,NaN


In [109]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Tr', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Tr Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [110]:
# distribution
(
    ggplot(result_data, aes(x='diff_Tr'))
    + geom_histogram()
    + ylab('Tr Diff [ms]')
)

In [111]:
Tr_stat = pg.mixed_anova(dv='diff_Tr',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Tr_stat

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,group,3589.260417,1,22,3589.260417,1.681529,0.208158,NaN,0.071006,NaN,NaN,NaN,NaN
1,measurement,1113.531250,3,66,371.177083,0.333728,0.800977,0.748283,0.014943,0.805833,False,0.574002,0.034171
2,Interaction,479.114583,3,66,159.704861,0.143592,0.933427,NaN,0.006485,NaN,NaN,NaN,NaN


In [112]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Vc', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Vc Difference [mm/ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [113]:
# distribution
(
    ggplot(result_data, aes(x='diff_Vc'))
    + geom_histogram()
    + ylab('Vc Diff [mm/ms]')
)

In [114]:
Vc_stat = pg.mixed_anova(dv='diff_Vc',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Vc_stat

C:\Users\bvenn\AppData\Roaming\Python\Python311\site-packages\pingouin\distribution.py:1004: RuntimeWarning: divide by zero encountered in scalar divide
  W = np.prod(eig) / (eig.sum() / d) ** d


,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.000119,1,22,0.000119,0.128531,0.723377,0.005808,NaN
1,measurement,0.000885,3,66,0.000295,1.397414,0.251395,0.059725,0.931854
2,Interaction,0.000147,3,66,0.000049,0.232779,0.873229,0.010470,NaN


## Plot for TMG parameter definition 

In [115]:
verlauf = pd.read_excel(r"C:\Users\bvenn\OneDrive\Desktop\TMG Graphik.xlsx")
verlauf

,20_mA,30_mA,40_mA,50_mA,60_mA,70_mA,80_mA,90_mA,100_mA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000520,0.002236,0.003270,0.003492,0.003988,0.003658,0.003437,0.003407,0.003223
2,0.000295,0.001931,0.003045,0.003284,0.004086,0.003511,0.003593,0.003363,0.003097
3,-0.000094,0.001296,0.002376,0.002591,0.003767,0.002862,0.003419,0.002877,0.002588
4,-0.000653,0.000340,0.001257,0.001401,0.003021,0.001687,0.002919,0.001936,0.001714
...,...,...,...,...,...,...,...,...,...
995,-0.010347,-0.010413,-0.004649,0.001597,0.002891,0.005929,0.004153,0.009029,-0.000809
996,-0.004926,-0.004960,-0.002213,0.000744,0.001377,0.002819,0.001974,0.004286,-0.000427
997,-0.001939,-0.001954,-0.000871,0.000287,0.000543,0.001108,0.000776,0.001683,-0.000182
998,-0.000585,-0.000590,-0.000263,0.000085,0.000164,0.000334,0.000234,0.000507,-0.000059


In [116]:
verlauf['index'] = verlauf.index
melted_verlauf = pd.melt(frame=verlauf, id_vars='index', var_name='ampere', value_name='value')
melted_verlauf

,index,ampere,value
0,0,20_mA,0.000000
1,1,20_mA,0.000520
2,2,20_mA,0.000295
3,3,20_mA,-0.000094
4,4,20_mA,-0.000653
...,...,...,...
8995,995,100_mA,-0.000809
8996,996,100_mA,-0.000427
8997,997,100_mA,-0.000182
8998,998,100_mA,-0.000059


In [117]:
(
    ggplot(melted_verlauf, aes(x='index', y='value', color='ampere'))
    + geom_point(size=1.2)
    + xlab('Displacement [mm]')
)

In [118]:
(
    ggplot(melted_verlauf, aes(x='index', y='value'))
    + geom_point(size=1.2)
    + ylab('Displacement [mm]')
    + xlab('Time [ms]')
    + labs(title='Incremental TMG Protocol. 20 - 100 mA Stimulus')
)

In [119]:
(
    ggplot(verlauf.iloc[0:426], aes(x='index', y='90_mA'))
    + geom_point(size=1.8)
    + ylab('Displacement [mm]')
    + xlab('Time [ms]')
    + labs(title='TMG-Parameter Definition')
)